<a href="https://colab.research.google.com/github/LC1332/Speaker-Grouping/blob/main/notebook/%E6%89%B9%E9%87%8F%E6%8A%BD%E5%8F%96%E5%9B%BE%E7%89%87%E5%92%8C%E9%9F%B3%E9%A2%91%E7%89%B9%E5%BE%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 载入并解压数据
- [ ] 批量抽取clip中的vit特征
- [ ] 批量抽取音频特征
- [ ] 保存特征为parquet文件

# 载入数据并解压

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Speaker/data

liangjian_10.zip


In [3]:
zip_name = "/content/drive/MyDrive/Speaker/data/liangjian_10.zip"

我希望将 zip_name = "/content/drive/MyDrive/Speaker/data/liangjian_10.zip"

这个文件copy到/content/下 并且解压到/content/{zip_file_name}

要如何用python实现

In [4]:
import shutil
import zipfile
import os

# 定义源 ZIP 文件路径和目标路径
zip_path = "/content/drive/MyDrive/Speaker/data/liangjian_10.zip"
target_dir = "/content/"

# 复制 ZIP 文件到指定目录
shutil.copy(zip_path, target_dir)

# 构建解压后的目标文件夹路径
zip_file_name = os.path.basename(zip_path)
unzip_folder = os.path.join(target_dir, zip_file_name.replace('.zip', ''))

# 创建目标文件夹
if not os.path.exists(unzip_folder):
    os.makedirs(unzip_folder)

# 解压 ZIP 文件
with zipfile.ZipFile(os.path.join(target_dir, zip_file_name), 'r') as zip_ref:
    zip_ref.extractall(unzip_folder)

print(f'文件已解压到 {unzip_folder}')


文件已解压到 /content/liangjian_10


In [5]:
!ls /content/liangjian_10/亮剑10

audios	meta.jsonl  screeshots


In [6]:
source_dir = "/content/liangjian_10/亮剑10"

# 解析meta

In [7]:
import pandas as pd

# 定义存有 JSONL 文件的目录
source_dir = "/content/liangjian_10/亮剑10"
jsonl_file = f"{source_dir}/meta.jsonl"

# 读取 JSONL 文件到 pandas DataFrame
# lines=True 表示文件中的每一行都是一个独立的 JSON 对象
meta_data = pd.read_json(jsonl_file, lines=True)

# 显示前 5 行数据
print(meta_data.head())


   人物      人物台词          开始时间          结束时间                 audio_file  \
0  段鹏  八路打鬼子不含糊  00:01:34.140  00:01:35.340  outdata/亮剑10/audios/0.wav   
1  段鹏      可俺不行  00:01:35.340  00:01:37.060  outdata/亮剑10/audios/1.wav   
2  段鹏    俺家里有老娘  00:01:37.060  00:01:38.340  outdata/亮剑10/audios/2.wav   
3  段鹏       俺走了  00:01:38.340  00:01:39.320  outdata/亮剑10/audios/3.wav   
4  段鹏      俺娘咋办  00:01:39.320  00:01:40.100  outdata/亮剑10/audios/4.wav   

                  screeshot_file  
0  outdata/亮剑10/screeshots/0.jpg  
1  outdata/亮剑10/screeshots/1.jpg  
2  outdata/亮剑10/screeshots/2.jpg  
3  outdata/亮剑10/screeshots/3.jpg  
4  outdata/亮剑10/screeshots/4.jpg  


# 抽取图片特征

In [8]:
def clean_name( file_name ):
    return file_name.replace("outdata/亮剑10/", "")

In [9]:
!wget -q https://raw.githubusercontent.com/LC1332/simple-face-recognition/main/CLIPExtractor.py

In [12]:
screenshots = meta_data["screeshot_file"].values
screenshots = [clean_name(f) for f in screenshots]
screenshots = [os.path.join(source_dir, f) for f in screenshots]

print(screenshots[:5])

['/content/liangjian_10/亮剑10/screeshots/0.jpg', '/content/liangjian_10/亮剑10/screeshots/1.jpg', '/content/liangjian_10/亮剑10/screeshots/2.jpg', '/content/liangjian_10/亮剑10/screeshots/3.jpg', '/content/liangjian_10/亮剑10/screeshots/4.jpg']


In [13]:
from CLIPExtractor import CLIPExtractor

extractor = CLIPExtractor()

config.json:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [14]:
visual_feature = extractor.extract(screenshots)

100%|██████████| 47/47 [00:17<00:00,  2.66it/s]


---

# 抽取音频特征

In [17]:
!wget -q https://raw.githubusercontent.com/LC1332/Legacy-Haruhi-1.0/main/yuki_builder/audio_feature_ext/audio_fea_ext.py